In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mimg

import ipywidgets as ipw
import numpy as np

from astropy.nddata import CCDData, block_reduce
from astropy.visualization import AsymmetricPercentileInterval, make_lupton_rgb

from astrowidgets.bqplot import ImageWidget

# Another color image maker

In [ ]:
image_widgets = dict(
    red=ImageWidget(),
    green=ImageWidget(),
    blue=ImageWidget()
)

In [ ]:
red = CCDData.read('combined_light_filter_rp.fit')
greenish = CCDData.read('combined_light_filter_gp.fit')
blue = CCDData.read('combined_light_filter_B.fit')

In [ ]:
reduce_fac = 8
red_sm = block_reduce(red.data, reduce_fac)
green_sm = block_reduce(greenish.data, reduce_fac)
blue_sm = block_reduce(blue.data, reduce_fac)

In [ ]:
image_widgets['red'].load_array(red_sm)
image_widgets['green'].load_array(green_sm)
image_widgets['blue'].load_array(blue_sm)

In [ ]:
def make_slider():
    slider = ipw.FloatRangeSlider(min=0, max=100, step=0.1, 
                                  description='Set black and white',
                                  style={'description_width': 'initial'},
                                  continuous_update=False,
                                  layout={'width': '100%'}
                                 )
    return slider

In [ ]:
level_sliders = dict(
    red=make_slider(),
    green=make_slider(),
    blue=make_slider()
)

In [ ]:
def make_observer(color):
    def observer(change):
        minval, maxval = change['new']
        image_widgets[color].cuts = AsymmetricPercentileInterval(minval, maxval)
        
    return observer

## 1. Adjust each of the combined image (r, g, b) so that the background is black and you can see the detail you want

In [ ]:
tab_set = ipw.Tab()
kids = []
boxes = {}
for idx, color in enumerate(['red', 'green', 'blue']):
    boxes[color] = ipw.VBox(children=[level_sliders[color], image_widgets[color]])
    this_observer = make_observer(color)
    level_sliders[color].observe(this_observer, names='value')
    this_observer(dict(new=level_sliders[color].value))
    tab_set.set_title(idx, color)
    kids.append(boxes[color])


tab_set.children = kids
tab_set

## Run the cells below to save red, green, blue as png

In [ ]:
def save_png(x, color):
    shape = np.array(x.shape)
    # Reverse order for reasons I do not understand...
    shape = shape[::-1]
    # Replace NaNs with black pixels
    x = np.nan_to_num(x)
    mimg.imsave(f'{color}.png', x, format='png', cmap='gray')
    
def save_all():
    x = image_widgets['red'].cuts(red.data)
    save_png(x, 'red')
    x = image_widgets['green'].cuts(greenish.data)
    save_png(x, 'green')
    x = image_widgets['blue'].cuts(blue.data)
    save_png(x, 'blue')

In [ ]:
save_all()

In [ ]:
plt.figure(figsize=(10, 10))
comb = np.zeros(list(red_sm.shape) + [3])
def quick_color_rgb(r=0.5, g=0.5, b=0.5):
    red_sc = r * image_widgets['red'].cuts(red_sm) 
    green_sc = g * image_widgets['green'].cuts(green_sm)
    blue_sc = b * image_widgets['blue'].cuts(blue_sm)
    comb[:, :, 0] = red_sc
    comb[:, :, 1] = green_sc
    comb[:, :, 2] = blue_sc
    plt.figure(figsize=(8, 8))
    max_img = np.nanmax(comb)
    plt.title(f'{max_img:.3f} {r=:.2f} {g=:.2f} {b=:.2f}')
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    plt.imshow(comb/max_img)

## Adjust the contribution of the red, green and blue images to the final image

In [ ]:
ranges = (0, 1, 0.01)
ipw.interact(quick_color_rgb, r=ranges, g=ranges, b=ranges)

In [ ]:
def full_res(r, g, b):
    plt.figure(figsize=(20, 20))
    red_sc = r * image_widgets['red'].cuts(red.data) 
    green_sc = g * image_widgets['green'].cuts(greenish.data)
    blue_sc = b * image_widgets['blue'].cuts(blue.data)
    comb = np.zeros(list(red.data.shape) + [3])
    comb[:, :, 0] = red_sc
    comb[:, :, 1] = green_sc
    comb[:, :, 2] = blue_sc
    max_img = np.nanmax(comb)
    plt.imshow(comb/max_img)
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)

In [ ]:
full_res(.51, 0.38, 0.55)


In [ ]:
plt.savefig('color.png')